In [1]:
import json
from pathlib import Path
from functools import partial

In [3]:
import tensorflow as tf

In [4]:
from spleeter.audio.adapter import get_default_audio_adapter
from spleeter.separator import Separator
from spleeter.audio.convertor import to_stereo
from spleeter.model import model_fn
from spleeter.model.provider import ModelProvider
from spleeter.dataset import get_training_dataset, get_validation_dataset

In [5]:
config_path = "config/voice_config.json"

In [6]:
with open(config_path) as f:
    params = json.load(f)

#### Создаем `estimator`

In [7]:
def _create_estimator(params):
    """ Creates estimator.

    :param params: TF params to build estimator from.
    :returns: Built estimator.
    """
    session_config = tf.compat.v1.ConfigProto()
    session_config.gpu_options.per_process_gpu_memory_fraction = 0.45
    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir=params['model_dir'],
        params=params,
        config=tf.estimator.RunConfig(
            save_checkpoints_steps=params['save_checkpoints_steps'],
            tf_random_seed=params['random_seed'],
            save_summary_steps=params['save_summary_steps'],
            session_config=session_config,
            log_step_count_steps=10,
            keep_checkpoint_max=2))
    return estimator

In [8]:
estimator = _create_estimator(params)
estimator

INFO:tensorflow:Using config: {'_model_dir': 'voice_model', '_tf_random_seed': 3, '_save_summary_steps': 5, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.45
}
, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020676795388>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


#### Загрузчик тренировочных данных

In [9]:
def _create_train_spec(params, audio_adapter, audio_path):
    """ Creates train spec.

    :param params: TF params to build spec from.
    :returns: Built train spec.
    """
    input_fn = partial(get_training_dataset, params, audio_adapter, audio_path)
    train_spec = tf.estimator.TrainSpec(
        input_fn=input_fn,
        max_steps=params['train_max_steps'])
    return train_spec

In [10]:
audio_adapter = get_default_audio_adapter()

In [11]:
audio_path = str(Path('.'))

In [12]:
train_spec = _create_train_spec(params, audio_adapter, audio_path)

#### Загрузчик валидационных данных

In [13]:
def _create_evaluation_spec(params, audio_adapter, audio_path):
    """ Setup eval spec evaluating ever n seconds

    :param params: TF params to build spec from.
    :returns: Built evaluation spec.
    """
    input_fn = partial(
        get_validation_dataset,
        params,
        audio_adapter,
        audio_path)
    evaluation_spec = tf.estimator.EvalSpec(
        input_fn=input_fn,
        steps=None,
        throttle_secs=params['throttle_secs'])
    return evaluation_spec

In [14]:
evaluation_spec = _create_evaluation_spec(params, audio_adapter, audio_path)

#### Обучение
(вся магия — внутри)

In [ ]:
tf.estimator.train_and_evaluate(estimator, train_spec, evaluation_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 300 or save_checkpoints_secs None.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

INFO:tensorflow:Calling model_fn.

INFO:tensorflow:Apply unet for voice_spectrogram
INFO:tensorflow:Apply unet for noise_spectrogram
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into voice_model\model.ckpt.


INFO:spleeter:Loading audio b'.\\..\\..\\data\\nr\\train\\mix\\11faafcdf712444880d3964e62b3c84d\\mix.wav' from 0.0 to 20.0


INFO:spleeter:Loading audio b'.\\..\\..\\data\\nr\\train\\mix\\0dc3b36938a341749c05a5355dd4c1f2\\mix.wav' from 0.0 to 20.0


INFO:spleeter:Loading audio b'.\\..\\..\\data\\nr\\train\\mix\\60fbc287eaea4600bb1d8fb88eb62430\\mix.wav' from 0.0 to 20.0
INFO:spleeter:Loading audio b'.\\..\\..\\data\\nr\\train\\mix\\35b4f36726354da3b902fc885fa858a7\\mix.wav' from 0.0 to 20.0
INFO:spleeter:Audio data loaded successfully
INFO:spleeter:Audio data loaded successfully
INFO:spleeter:Loading audio b'.\\..\\..\\data\\nr\\train\\mix\\2228b51a7a68483eaeef283a5cf681b9\\mix.wav' from 0.0 to 20.0
INFO:spleeter:Audio data loaded successfully
INFO:spleeter:Loading audio b'.\\..\\..\\data\\nr\\train\\mix\\14f14d62c9ae402faf8c0e166d054c0c\\mix.wav' from 0.0 to 20.0
INFO:spleeter:Audio data loaded successfully
INFO:spleeter:Loading audio b'.\\..\\..\\data\\nr\\train\\mix\\a143fd4cc7594600b0f939f4f67cb5b6\\mix.wav' from 0.0 to 20.0
INFO:spleeter:Loading audio b'.\\..\\..\\data\\nr\\train\\source\\35b4f36726354da3b902fc885fa858a7\\1.wav' from 0.0 to 20.0
INFO:spleeter:Loading audio b'.\\..\\..\\data\\nr\\train\\mix\\570cc46328994bd8b7

#### Сохраним полученные веса

In [ ]:
ModelProvider.writeProbe(params['model_dir'])